In [1]:
import ee

from src.controllers.plotter_base import PhenologyPlotter
from src.controllers.metrics_vos_pos import VosPosMetrics
from src.controllers.metrics_bos_eso import BosEosMetrics
from src.controllers.sentinel_ import  Sentinel2Processor
from src.controllers.geometry import ProcessadorGeoDataFrame
from src.controllers.metrics_geometrics import PhenologyMetrics
from src.controllers.time_series import VegetationIndexProcessor
from src.controllers.eras import PrecipitationTemperatureRadiationData

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-carbonei')

In [2]:
####Geometries variables####

#Paht polygons
path = r'data\ms_field_boundaries.gpkg'

#Index polygon, defoult is 0
index_poligon = 16

####Datas de análise####
#Start date analises 
start_date = '2023-01-01'

#End date analises
end_date = '2023-12-30'


####Time series parameters####
#ndvi_value to NDVI and evi_value to EVI 
vegetation_index = 'ndvi_value'

#Threshould index to NDVI recomend 0.37
threshould_index = 0.3

####Savitzky-Golay to smoothing the vegetation index####
# Window to smooth Savitzky-Golay
window_size = 5

# Type of polynomial 
poly_order = 2

####VOS and POS metric####
#This value define the window to get VOS and POS, by experience use 3 parts os the full crop season
order = 30

In [3]:
#Read geometry file
processador = ProcessadorGeoDataFrame(path)
vertices, geometry = processador.extrair_coordenadas(index_poligon)

In [4]:
# Convert points to a polygon
polygon = ee.Geometry.Polygon(vertices)

In [5]:
s2 = Sentinel2Processor(start_date, end_date, polygon)
df = s2.process_data()

processor = VegetationIndexProcessor(df, vegetation_index, window_size=window_size, poly_order=poly_order)
df = processor.process()

#Get and VOS and POS metrics
vos_pos_analyzer = VosPosMetrics(df , order)
phenology_df = vos_pos_analyzer.analyze_phenology()

#Get and BOS and EOS metrics
analysis = BosEosMetrics(df , phenology_df, threshould_index)
phenology_df  = analysis.execute_analysis()

analysis_metrics = PhenologyMetrics(phenology_df, df)
phenology_df  = analysis_metrics.derivate_metrics()

# Usage
lat = geometry.getInfo()['coordinates'][0][0][1]  # Exemplo de latitude
lon = geometry.getInfo()['coordinates'][0][0][0]  # Exemplo de longitude

data = PrecipitationTemperatureRadiationData(lat, lon, start_date, end_date)
df_01 = data.get_dataframe()


c:\Users\ander\OneDrive\Área de Trabalho\msu\msu_timeseries_ndvi\src\controllers\sentinel_.py:193: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_filtered[['ndvi_value', 'evi_value']] = df_filtered[['ndvi_value', 'evi_value']].applymap(
c:\Users\ander\OneDrive\Área de Trabalho\msu\msu_timeseries_ndvi\src\controllers\time_series.py:28: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  self.df = self.df.resample('D').interpolate(method='linear')
c:\Users\ander\OneDrive\Área de Trabalho\msu\msu_timeseries_ndvi\src\controllers\metrics_bos_eso.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [6]:
import pandas as pd
merged_df = pd.merge(df, df_01, left_on='timestamps', right_on='date', how='inner')


In [7]:
plotter = PhenologyPlotter(df, phenology_df, vegetation_index)
fig = plotter.plot_data()

#fig_1 = plotter.plot_data_01()

fig_precipitation = data.plot_precipitation()
fig_temperature = data.plot_temperature()
fig_radiation = data.plot_radiation()


fig.show()
#fig_1.show()
# Show the interactive plots
fig_precipitation.show()
fig_temperature.show()
fig_radiation.show()